In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
from datetime import datetime
%matplotlib inline
pd.set_option('display.max_rows', 200)

In [2]:
all_loans = pd.read_csv('./data/loans-kaggle.csv', low_memory=False)

In [3]:
all_loans.shape

(887379, 74)

In [4]:
loans = all_loans.copy()

In [5]:
loans.columns

Index([u'id', u'member_id', u'loan_amnt', u'funded_amnt', u'funded_amnt_inv',
       u'term', u'int_rate', u'installment', u'grade', u'sub_grade',
       u'emp_title', u'emp_length', u'home_ownership', u'annual_inc',
       u'verification_status', u'issue_d', u'loan_status', u'pymnt_plan',
       u'url', u'desc', u'purpose', u'title', u'zip_code', u'addr_state',
       u'dti', u'delinq_2yrs', u'earliest_cr_line', u'inq_last_6mths',
       u'mths_since_last_delinq', u'mths_since_last_record', u'open_acc',
       u'pub_rec', u'revol_bal', u'revol_util', u'total_acc',
       u'initial_list_status', u'out_prncp', u'out_prncp_inv', u'total_pymnt',
       u'total_pymnt_inv', u'total_rec_prncp', u'total_rec_int',
       u'total_rec_late_fee', u'recoveries', u'collection_recovery_fee',
       u'last_pymnt_d', u'last_pymnt_amnt', u'next_pymnt_d',
       u'last_credit_pull_d', u'collections_12_mths_ex_med',
       u'mths_since_last_major_derog', u'policy_code', u'application_type',
       u'annu

### We are trying to predict whether a loan will default or not. We normalize the target variable to 0 or 1 so it becomes a binary classification.

In [6]:
loans.loan_status.value_counts()

Current                                                601779
Fully Paid                                             207723
Charged Off                                             45248
Late (31-120 days)                                      11591
Issued                                                   8460
In Grace Period                                          6253
Late (16-30 days)                                        2357
Does not meet the credit policy. Status:Fully Paid       1988
Default                                                  1219
Does not meet the credit policy. Status:Charged Off       761
Name: loan_status, dtype: int64

In [7]:
loans['defaulted'] = loans.loan_status.map({'Fully Paid': 0,
                                            'Current': 0,
                                            'Charged Off': 1,
                                            'Late (31-120 days)': 1,
                                            'Issued': 0,
                                            'In Grace Period': 0,
                                            'Late (16-30 days)': 0,
                                            'Does not meet the credit policy. Status:Fully Paid': 0,
                                            'Does not meet the credit policy. Status:Charged Off': 1,
                                            'Default': 1
                                           })
loans.drop('loan_status', axis=1, inplace=True)

### Lots of null values. Let's clean up the data

In [8]:
null_value_count = loans.isnull().sum().sort_values(ascending=False)
null_value_count[null_value_count != 0]

dti_joint                      886870
verification_status_joint      886868
annual_inc_joint               886868
il_util                        868762
mths_since_rcnt_il             866569
inq_fi                         866007
total_cu_tl                    866007
open_acc_6m                    866007
open_il_6m                     866007
open_il_12m                    866007
inq_last_12m                   866007
open_il_24m                    866007
open_rv_12m                    866007
open_rv_24m                    866007
max_bal_bc                     866007
all_util                       866007
total_bal_il                   866007
desc                           761350
mths_since_last_record         750326
mths_since_last_major_derog    665676
mths_since_last_delinq         454312
next_pymnt_d                   252971
total_rev_hi_lim                70276
tot_cur_bal                     70276
tot_coll_amt                    70276
emp_title                       51457
last_pymnt_d

Joint applications were introduced late in 2015. For the vast majority of individual applications, set joint_dti to 0, annual_inc_joint to 0, and verification_status_joint to 'Not applicable'

In [9]:
loans[loans.application_type == 'JOINT'].issue_d.value_counts()

Dec-2015    250
Nov-2015    187
Oct-2015     74
Name: issue_d, dtype: int64

In [10]:
loans[(loans.application_type == 'JOINT') & loans.dti_joint.isnull() & loans.annual_inc_joint.isnull()
     & loans.verification_status_joint.isnull()].shape

(0, 74)

In [11]:
loans['dti_joint'].fillna(value=0.0, inplace=True)
loans['annual_inc_joint'].fillna(value=0.0, inplace=True)
loans['verification_status_joint'].fillna(value='Not Applicable', inplace=True)

These fields were added on December 2015 and only exist for new loans. We can just drop them.

In [12]:
dec_2015_columns = ['il_util', 'mths_since_rcnt_il', 'open_acc_6m', 'inq_last_12m', 'open_il_6m', 'open_il_12m',
                   'open_il_24m', 'total_bal_il', 'open_rv_12m', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
                   'inq_fi', 'total_cu_tl', 'all_util']
loans = loans.drop(dec_2015_columns, axis=1)

Description is missing for most loans and since we are not doing any keyword or sentiment analyis, we can drop it.

In [13]:
loans = loans.drop('desc', axis=1)

Public records are derogatory records such as bankruptcy, civil judgment, and tax liens. These values are missing because these loans have no negative public records. Let's set the value to a very high number, i.e., 1,000,000

In [14]:
loans[loans.mths_since_last_record.isnull() & (loans.pub_rec > 0)].shape

(0, 59)

In [15]:
loans.mths_since_last_record.fillna(value=1000000, inplace=True)

Derogatories are public records or loans which have been delinquent for more than 90 days. mths_since_last_major_derog is the number of months since the last derogatory record. Since it's missing for most loans, we set it to the max of the column.

In [16]:
loans['mths_since_last_major_derog'].fillna(value=loans.mths_since_last_major_derog.max(), inplace=True)

mths_since_last_delinq is the number of months since last delinquency. If they currently have a delinquent account or one in the past two years, we set it to 0. Otherwise set it to max of column.

In [17]:
max_months = loans.mths_since_last_delinq.max()
def impute_mths_since_last_delinq(row):  
    months = max_months
    if row.acc_now_delinq > 0 or row.delinq_2yrs > 0:
        months = 0
    return months

loans.loc[loans.mths_since_last_delinq.isnull(), 'mths_since_last_delinq'] = \
    loans[loans.mths_since_last_delinq.isnull()].apply(impute_mths_since_last_delinq, axis=1)

For the following, we will impute using the median
tot_cur_bal => Total current balance of all accounts
total_rev_hi_lim => Total revolving credit limit
tot_coll_amt => Total collection amounts ever owed
revol_util => Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.
collections_12_mths_ex_med => Number of collections in 12 months excluding medical collections.

In [18]:
loans.tot_cur_bal.fillna(loans.tot_cur_bal.median(), inplace=True)
loans.total_rev_hi_lim.fillna(loans.total_rev_hi_lim.median(), inplace=True)
loans.tot_coll_amt.fillna(loans.tot_coll_amt.median(), inplace=True)
loans.revol_util.fillna(loans.revol_util.median(), inplace=True)
loans.collections_12_mths_ex_med.fillna(loans.collections_12_mths_ex_med.median(), inplace=True)

Since there are so many different employment titles, we cannot include it as a categorical feature. So we drop it.

In [19]:
loans.emp_title.value_counts().count()

299272

In [20]:
loans.drop('emp_title', axis=1, inplace=True)

These 29 loans issued in the summer of 2007 when Lending Club's screening process was still being implemented. We can simply remove these loans.

In [21]:
loans[loans.acc_now_delinq.isnull() & loans.total_acc.isnull() & loans.pub_rec.isnull() & loans.open_acc.isnull()
     & loans.inq_last_6mths.isnull() & loans.delinq_2yrs.isnull()].shape[0]

29

In [22]:
loans[loans.acc_now_delinq.isnull() & loans.total_acc.isnull() & loans.pub_rec.isnull() & loans.open_acc.isnull()
     & loans.inq_last_6mths.isnull() & loans.delinq_2yrs.isnull()].issue_d.value_counts()

Jun-2007    21
Aug-2007     4
Jul-2007     4
Name: issue_d, dtype: int64

In [23]:
missing_summer_2007 = ['delinq_2yrs', 'acc_now_delinq', 'inq_last_6mths', 'open_acc', 'pub_rec', 'total_acc']
loans.dropna(subset=missing_summer_2007, inplace=True)

Loans ID, Member ID and URL are unique to each loan, so we can drop those columns

In [24]:
loans.drop(['id', 'member_id', 'url'], axis=1, inplace=True)

We can drop title since we already have purpose as a categorical feature.

In [25]:
loans.drop('title', axis=1, inplace=True)

Since sub_grade encapsulates already captures grade, we can drop it.

In [26]:
loans.drop('grade', axis=1, inplace=True)

Drop all the columns that are dates.

In [27]:
date_columns = ['last_pymnt_d', 'earliest_cr_line', 'next_pymnt_d', 'last_credit_pull_d', 'issue_d']
loans.drop(date_columns, axis=1, inplace=True)

Employment length is ordinal. We convert it integers 0 to 10. For n/a, we assume that if the user left it blank, they are self employed or have other sources of income. So we map it 0.

In [28]:
loans.emp_length.value_counts()

10+ years    291566
2 years       78869
< 1 year      70587
3 years       70024
1 year        57092
5 years       55704
4 years       52529
n/a           44825
7 years       44594
8 years       43955
6 years       42949
9 years       34656
Name: emp_length, dtype: int64

In [29]:
def fix_employment_length(x):
    years = 0
    if x == 'n/a' or x[:1] == '<':
        years = 0
    elif x[:3] == '10+':
        years = 10
    else:
        years = int(x[:1])
    return years

loans['emp_length'] = loans.emp_length.apply(fix_employment_length)

We'll drop zip code, but keep state as a categorical feature

In [30]:
loans.drop('zip_code', axis=1, inplace=True)

Lending Club generally sells fractional notes of whole loans. But they also sell whole loans to institutional investors who request it. initial_list_status captures that as 'w' or 'f'. Since, it's not relevant to the borrower, we drop it.

In [31]:
loans.drop('initial_list_status', axis=1, inplace=True)

Policy code is 1 for all the loans in this dataset, so we can drop it.

In [32]:
loans.drop('policy_code', axis=1, inplace=True)

### Create dummy variables for the categorical features

In [33]:
loans.columns

Index([u'loan_amnt', u'funded_amnt', u'funded_amnt_inv', u'term', u'int_rate',
       u'installment', u'sub_grade', u'emp_length', u'home_ownership',
       u'annual_inc', u'verification_status', u'pymnt_plan', u'purpose',
       u'addr_state', u'dti', u'delinq_2yrs', u'inq_last_6mths',
       u'mths_since_last_delinq', u'mths_since_last_record', u'open_acc',
       u'pub_rec', u'revol_bal', u'revol_util', u'total_acc', u'out_prncp',
       u'out_prncp_inv', u'total_pymnt', u'total_pymnt_inv',
       u'total_rec_prncp', u'total_rec_int', u'total_rec_late_fee',
       u'recoveries', u'collection_recovery_fee', u'last_pymnt_amnt',
       u'collections_12_mths_ex_med', u'mths_since_last_major_derog',
       u'application_type', u'annual_inc_joint', u'dti_joint',
       u'verification_status_joint', u'acc_now_delinq', u'tot_coll_amt',
       u'tot_cur_bal', u'total_rev_hi_lim', u'defaulted'],
      dtype='object')

In [35]:
loans.shape

(887350, 45)

In [34]:
categorical_features = ['term', 'sub_grade', 'home_ownership', 'verification_status', 'pymnt_plan', 'purpose',
                       'addr_state', 'application_type', 'verification_status_joint']
